In [17]:
import requests
import pandas as pd

# get_wine_data(wine_id, year, page) 
# id year page 받아서 api에서 데이터 받아오기 함수
def get_wine_data(wine_id, year, page):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    }

    api_url = "https://www.vivino.com/api/wines/{id}/reviews?per_page=50&year={year}&page={page}"  # <-- increased the number of reviews to 9999

    data = requests.get(
        api_url.format(id=wine_id, year=year, page=page), headers=headers
    ).json()

    return data
#https://www.vivino.com/api/wines/2468359/reviews?per_page=50&year=2020&page=page
# 
r = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params={
        "country_code": "USA",
        #"country_codes[]": "pt",
        #"currency_code": "EUR",
        #"grape_filter": "varietal",
        "min_rating": "1",
        "order_by": "price",
        "order": "asc",
        #"page": 1,
        "price_range_max": "500",
        "price_range_min": "0",
        "wine_type_ids[]": "1",

        
#         "currency_code":"USA",
#             "min_rating":"1",
#             "order_by":"price",
#             "order":"asc",
#             "page": 1,
#             "wine_type_ids[]":"1"
    },
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    },
)

results = [
    (
        t["vintage"]["wine"]["winery"]["name"],
        t["vintage"]["year"],
        t["vintage"]["wine"]["id"],
        f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
        t["vintage"]["seo_name"],#Wine(eng) 영어이름 위에 이름은 불어등이 불러와짐. 
        t["vintage"]["statistics"]["ratings_average"],
        t["vintage"]["statistics"]["ratings_count"],
        t["vintage"]["image"]["variations"], #["bottle_large"], 추후에 뽑아서 자르는 것 필요
        t["vintage"]["wine"]['taste']['structure']['acidity'],
        t["vintage"]["wine"]['taste']['structure']['fizziness'],
        t["vintage"]["wine"]['taste']['structure']['intensity'],
        t["vintage"]["wine"]['taste']['structure']['sweetness'],
        t["vintage"]["wine"]['taste']['structure']['tannin'],
        t["vintage"]["wine"]['taste']['structure']['user_structure_count'],
        t["vintage"]["wine"]['taste']['structure']['calculated_structure_count'],
        t["vintage"]["wine"]['taste']['flavor'], # 추후 조금더 나눌 필요가 있지 않을까?
        t["vintage"]["wine"]['style']['description'],
        t["vintage"]["wine"]['style']['food'], #['name']  [안에 내용] 이것을 어떻게 처리할지 고민해봐야
        t["vintage"]["wine"]['style']['grapes'], # ['name'] 위와 같은 문제
        t["vintage"]["wine"]['style']['region'], #['name'] 지역정보가 없는 애들이 있어서 name을 찾지 못하는경우발생,
        t["vintage"]["wine"]['style']['region'], #['country']['name'], 같은 문제 발생 
        t["price"]["url"],
        #t["vintage"]["statistics"]["ratings_average"],
        #t["vintage"]["statistics"]["ratings_count"],
    )
    for t in r.json()["explore_vintage"]["matches"]
]

dataframe = pd.DataFrame(
    results,
    columns=["Winery", "Year", "Wine ID", "Wine","Winefullname(eng)", "Rating", "num_review",\
             "image", 'acidity', 'fizziness', 'intensity', 'sweetness', \
             'tannin', 'user_structure_count', 'calculated_structure_count', 'flavor', 'description', 'food', \
            'grapes','region','country', 'url' ], #'name2', 'mostusedgrapes',
)
# 와인데이터(dataframe)에 리뷰 합치기 
ratings = []
for _, row in dataframe.iterrows():
    page = 1
    while True:
        print(
            f'Getting info about wine {row["Wine ID"]}-{row["Year"]} Page {page}'
        )

        d = get_wine_data(row["Wine ID"], row["Year"], page)

        if not d["reviews"]:
            break

        for r in d["reviews"]:
            ratings.append(
                [
                    row["Year"],
                    row["Wine ID"],
                    r["rating"],
                    r["note"],
                    r["created_at"],
                ]
            )

        page += 1

ratings = pd.DataFrame(
    ratings, columns=["Year", "Wine ID", "User Rating", "Note", "CreatedAt"]
)

df_out = ratings.merge(dataframe)
df_out.to_csv("data.csv", index=False)

KeyError: 'explore_vintage'

In [4]:
import pandas as pd

df_out=pd.read_csv("data.csv")
df_out.head()

,Year,Wine ID,User Rating,Note,CreatedAt,Winery,Wine,Winefullname(eng),Rating,num_review,...,tannin,user_structure_count,calculated_structure_count,flavor,description,food,grapes,region,country,url
0,2016,1154146,4.0,Nice. Big with lots of tannin. Bought in Italy...,2018-08-16T18:31:10.000Z,Melini,San Lorenzo Chianti 2016,melini-san-lorenzo-chianti-2016,3.6,645,...,3.446,103,134,"[{'group': 'red_fruit', 'stats': {'count': 65,...","The most commonly consumed wine in Italy, and ...","[{'id': 4, 'name': 'Beef', 'background_image':...","[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...",http://www.winebow.co.kr/shop/goods/goods_view...
1,2016,1154146,3.5,Not the best Chianti i have ever had but consi...,2018-09-06T17:24:51.000Z,Melini,San Lorenzo Chianti 2016,melini-san-lorenzo-chianti-2016,3.6,645,...,3.446,103,134,"[{'group': 'red_fruit', 'stats': {'count': 65,...","The most commonly consumed wine in Italy, and ...","[{'id': 4, 'name': 'Beef', 'background_image':...","[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...",http://www.winebow.co.kr/shop/goods/goods_view...
2,2016,1154146,3.5,Packs more of a tannic punch than your average...,2018-07-08T17:37:24.000Z,Melini,San Lorenzo Chianti 2016,melini-san-lorenzo-chianti-2016,3.6,645,...,3.446,103,134,"[{'group': 'red_fruit', 'stats': {'count': 65,...","The most commonly consumed wine in Italy, and ...","[{'id': 4, 'name': 'Beef', 'background_image':...","[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...",http://www.winebow.co.kr/shop/goods/goods_view...
3,2016,1154146,4.0,Good balance of tannin and acid. Raspberries a...,2018-03-24T02:11:19.000Z,Melini,San Lorenzo Chianti 2016,melini-san-lorenzo-chianti-2016,3.6,645,...,3.446,103,134,"[{'group': 'red_fruit', 'stats': {'count': 65,...","The most commonly consumed wine in Italy, and ...","[{'id': 4, 'name': 'Beef', 'background_image':...","[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...",http://www.winebow.co.kr/shop/goods/goods_view...
4,2016,1154146,4.0,Smooth with blackberries on the nose. Very en...,2018-09-27T11:55:14.000Z,Melini,San Lorenzo Chianti 2016,melini-san-lorenzo-chianti-2016,3.6,645,...,3.446,103,134,"[{'group': 'red_fruit', 'stats': {'count': 65,...","The most commonly consumed wine in Italy, and ...","[{'id': 4, 'name': 'Beef', 'background_image':...","[{'id': 16, 'name': 'Sangiovese', 'seo_name': ...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...","{'id': 394, 'name': 'Toscana', 'name_en': 'Tus...",http://www.winebow.co.kr/shop/goods/goods_view...
